# Project 1 — Optimizing Default Model by First Payment Default

**Name:** Anthony Roca  
**Due:** Feb 18, 2026  
**Competition:** [Kaggle – Optimizing Default Model by First Payment Default](https://www.kaggle.com/competitions/optimizingdefaultmodelbyfirstpaymentdefault/overview)

---

## Workflow Summary

| Step | Approach |
|------|----------|
| **Stack** | NumPy, Pandas, Scikit-learn (plus SHAP for explainability) |
| **EDA** | Concise style similar to `labs/eda_lab/eda_lab.ipynb` — load/inspect, missing values, types, distributions, correlations, key visualizations. Markdown narrative after each code block. |
| **Splits** | Standard train / validation / test. |
| **Feature selection** | L1 and L2 regularization; Elastic Net for logistic regression. Fine-tune later if needed. |
| **Models** | Elastic Net (logistic), SVM, Random Forest, XGBoost. Fixed random seeds and **sklearn Pipelines** for valid comparison. |
| **Parameter optimization** | Grid search; deferred to the end (time-intensive). |
| **Class imbalance** | To be decided after EDA and data prep/cleaning. |
| **Global explainability** | SHAP (Shapley) analysis — plots plus written interpretation. |

---
## Setup & Imports

In [ ]:
import numpy as np
import pandas as pd
from sklearn import pipeline, model_selection, preprocessing, linear_model, svm, ensemble
# XGBoost and SHAP added when needed
RANDOM_STATE = 42

---
## 1. Exploratory Data Analysis (EDA)

Load and inspect the dataset; summarize missing values, feature types, distributions, correlations; add concise visualizations. Narrative in markdown after each code block.

---
## 2. Train / Validation / Test Split

Standard split; document method and proportions.

---
## 3. Feature Selection

L1/L2 regularization; Elastic Net for logistic regression. Refine later if needed.

---
## 4. Train Multiple Classification Models

Elastic Net, SVM, Random Forest, XGBoost — using fixed `RANDOM_STATE` and sklearn Pipelines. Compare on validation set.